In [38]:
import pandas as pd

In [39]:
data = pd.read_csv("COVID19MEXICO.csv")
data

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,RESULTADO_PCR,RESULTADO_PCR_COINFECCION,TOMA_MUESTRA_ANTIGENO,RESULTADO_ANTIGENO,CLASIFICACION_FINAL_COVID,CLASIFICACION_FINAL_FLU,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
0,2025-07-29,167f1a,1,12,1,2,1,1,3,1,...,997,997,2,97,6,6,99,México,97,97
1,2025-07-29,ga8a474,1,4,20,2,20,20,413,1,...,997,997,2,97,6,6,99,México,97,97
2,2025-07-29,gb733da,1,6,8,2,8,8,37,1,...,997,997,2,97,6,6,99,México,97,97
3,2025-07-29,g9ff7b3,1,4,32,1,32,32,17,2,...,5,5,2,97,7,7,99,México,97,2
4,2025-07-29,g90b5c8,1,6,10,1,10,10,5,2,...,5,5,2,97,7,7,99,México,97,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99521,2025-07-29,g897fca,1,3,21,1,21,21,53,2,...,999,999,2,97,6,6,99,México,97,2
99522,2025-07-29,ge40110,1,15,16,2,15,16,102,2,...,999,999,2,97,6,6,99,México,97,2
99523,2025-07-29,g528417,1,15,2,1,2,2,1,2,...,999,999,2,97,6,6,99,México,97,2
99524,2025-07-29,gab8b98,1,12,19,2,19,19,44,1,...,999,999,2,97,6,6,99,México,97,97


# Variables que voy a tomar y que significa:
En el contexto de la base de datos de la Secretaría de Salud de México, los códigos de valor son:


1: Sí


2: No


97, 98, 99: Desconocido o no aplica


Edad:    Numero de la edad del paciente


INTUBADO:	Paciente que fue o no intubado.


NEUMONIA:	Paciente que presentó o no diagnóstico de neumonía.


DIABETES:	Paciente con un diagnóstico previo de diabetes.


EPOC:	Paciente con un diagnóstico previo de enfermedad pulmonar obstructiva crónica.


ASMA:	Paciente con un diagnóstico previo de asma.


INMUSUPR:	Paciente con un diagnóstico previo de inmunosupresión.


HIPERTENSION:	Paciente con un diagnóstico previo de hipertensión.


CARDIOVASCULAR:	Paciente con un diagnóstico previo de enfermedad cardiovascular.


OBESIDAD:	Paciente con un diagnóstico previo de obesidad.


RENAL_CRONICA:	Paciente con un diagnóstico previo de enfermedad renal crónica.


TABAQUISMO:	Paciente con un diagnóstico previo de tabaquismo.


OTRO_CASO:	Indica si el paciente tuvo contacto con otro caso confirmado de COVID-19.


UCI:     Indica si el paciente requirió o no ser ingresado en una Unidad de Cuidados Intensivos

FECHA_DEF: Contiene la fecha del fallecimiento del paciente en caso de tener 9999-99-99 o algo parecido no ha muerto



Y la más Importante


### CLASIFICACION_FINAL_COVID


3: Negativo a SARS-CoV-2: El caso fue descartado tras obtener un resultado de laboratorio negativo.


4: Caso Descartado por Dictaminación: El caso fue descartado por un comité de expertos sin una prueba de laboratorio concluyente.


5: Sospechoso: Es el estado inicial y transitorio de un caso, cuya clasificación final está pendiente de confirmación o descarte.


6: Confirmado por Laboratorio: El caso fue confirmado mediante una prueba de laboratorio, típicamente RT-PCR, el método de mayor fiabilidad.


7: Positivo por Dictaminación o Asociación: El caso fue catalogado como positivo por un comité de expertos basándose en un nexo epidemiológico con un caso confirmado o en criterios clínicos, sin una prueba de laboratorio concluyente.

### CLASIFICACION_FINAL_FLU

(3): Caso confirmado


(7): Caso negativo


(4, 5 ó 6): Sin información (sin muestra o muestra inválida)

# Selección de variables y limpieza

In [40]:
# Seleccionando las columnas de interes
data = data[[
    "EDAD",
    "INTUBADO",
    "NEUMONIA",
    "DIABETES",
    "EPOC",
    "ASMA",
    "INMUSUPR",
    "HIPERTENSION",
    "CARDIOVASCULAR",
    "OBESIDAD",
    "RENAL_CRONICA",
    "TABAQUISMO",
    "OTRO_CASO",
    "UCI",
    "FECHA_DEF",
    "CLASIFICACION_FINAL_COVID",
    "CLASIFICACION_FINAL_FLU"
]]

# Lista de valores que representan datos faltantes
missing_codes = [97, 99, 997, 999, 9999, '97', '99', '997', '999', '9999', '9999-99-99']

# Reemplazar por 0 en todo el DataFrame
data.replace(missing_codes, 0)

# Crear una columana adulto mayor para centrarse solo en esa la probabilidad 
adulto_mayor = data["EDAD"].apply(
    lambda x: 1 if pd.notna(x) and x >= 65 else 0)
data.insert(1, 'AM', adulto_mayor)

def crear_columna_enfermo(df):
    df = df.copy()
    
    df['ENFERMO'] = (
        (df['CLASIFICACION_FINAL_COVID'].isin([6, 7])) |  # COVID positivo
        (df['CLASIFICACION_FINAL_FLU'] == 3)              # Influenza positivo
    )
    # Igual usamos 1 para casos positivos y 2 para negativos
    df['ENFERMO'] = df['ENFERMO'].map({True: 1, False: 2})
    
    return df
data = crear_columna_enfermo(data)
data

,EDAD,AM,INTUBADO,NEUMONIA,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,UCI,FECHA_DEF,CLASIFICACION_FINAL_COVID,CLASIFICACION_FINAL_FLU,ENFERMO
0,8,0,97,2,2,2,2,2,2,2,2,2,2,2,97,9999-99-99,6,6,1
1,23,0,97,2,2,2,2,2,2,2,2,2,2,2,97,9999-99-99,6,6,1
2,18,0,97,2,2,2,1,2,2,2,2,2,2,2,97,9999-99-99,6,6,1
3,24,0,2,2,2,2,1,2,2,2,2,2,2,2,2,9999-99-99,7,7,1
4,47,0,2,2,2,2,2,2,2,2,2,2,2,2,2,9999-99-99,7,7,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99521,83,1,2,2,2,2,2,2,1,2,2,2,2,2,2,9999-99-99,6,6,1
99522,50,0,2,1,1,2,2,2,1,2,2,1,2,1,2,9999-99-99,6,6,1
99523,13,0,2,2,2,2,2,2,2,2,2,2,2,2,2,9999-99-99,6,6,1
99524,37,0,97,2,2,2,2,2,2,2,2,2,2,1,97,9999-99-99,6,6,1


# Probabilidades de estar enfermo dado el sintoma

In [41]:
s = data["ENFERMO"].value_counts()
s

ENFERMO
1    92054
2     7472
Name: count, dtype: int64

In [42]:
columnas = ["AM", "INTUBADO", "NEUMONIA", "DIABETES", "EPOC", "ASMA", "INMUSUPR", "HIPERTENSION", "CARDIOVASCULAR", "OBESIDAD", "RENAL_CRONICA", "TABAQUISMO", "OTRO_CASO"]
for sintoma in columnas:
    # Contar cuantas veces coincide el sintoma con que este enfermo el paciente
    sintoma_y_enfermo = len(data[(data[sintoma] == 1) & (data['ENFERMO'] == 1)])
    total_con_sintoma = len(data[data[sintoma] == 1])
    
    #Calcular cuantas personas enfermas tienen el sintoma
    pct_enfermos_con_sintoma = (sintoma_y_enfermo / 92054) * 100
    # Calcular de las personas con el sintoma cuantos estan enfermos
    pct_sintoma_que_son_enfermos = (sintoma_y_enfermo / total_con_sintoma) * 100
        
    print(f"{sintoma:15} {pct_enfermos_con_sintoma:.2f}% {pct_sintoma_que_son_enfermos:10.2f}%")

AM              16.88%      91.01%
INTUBADO        2.22%      92.72%
NEUMONIA        23.73%      92.90%
DIABETES        13.10%      91.69%
EPOC            3.36%      92.46%
ASMA            3.97%      93.23%
INMUSUPR        3.07%      89.64%
HIPERTENSION    15.80%      91.34%
CARDIOVASCULAR  3.36%      92.25%
OBESIDAD        6.74%      92.09%
RENAL_CRONICA   3.43%      91.24%
TABAQUISMO      4.49%      92.09%
OTRO_CASO       18.23%      90.45%
